In [ ]:
using PyPlot, Random, ProgressMeter

In [ ]:
function sfth(x,t) #soft-thresholding
    sign(x) * max(0.0, abs(x) - t)
end

In [ ]:
"""
PGM (Algorithm 1) to train a two-layer neural net. m is the number of neurons.
"""
function neuralnet_PGM(X, Y, m, stepsize, niter; p=1, lambda=0.0)
    (n,d) = size(X) # n samples in R^d
    # initialize
    θ = randn(m, d)
    θ = θ ./ sqrt.(sum(θ.^2, dims=2)) # random neurons on the sphere
    θ[:,1] = cos.(range(0,2π,length=m))
    θ[:,2] = sin.(range(0,2π,length=m))
    a = zeros(m) # initialization
    gradG = copy(a)
    as = zeros(m,niter)
    loss  = zeros(niter)
    act  =  max.( θ * X', 0.0) # activations
    
    @showprogress 1 "Training neural network..." for iter = 1:niter
        as[:,iter] = a
        out   =  (1/m) * sum( a .* act , dims=1)[:] # predictions of the network
        loss[iter] = (1/2)*sum( ( out - Y).^2 )/n + lambda * sum(abs.(a))/m
        gradR = (out .- Y)/n  # size n
        gradG = act * gradR
        if p>1
            temp = sfth.( sign.(a).*abs.(a).^(p-1) /(p-1) - stepsize * gradG, stepsize*lambda)
            a    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
        else
            a = sinh.(sfth.( asinh.(a) - stepsize * gradG, stepsize*lambda))
        end
    end

    return as,θ, loss, gradG
end

In [ ]:
"""
APGM (Algorithm 1) to train a two-layer neural net. m is the number of neurons.
"""
function neuralnet_APGM(X, Y, m, stepsize, niter; p=1, lambda=0.0)
    (n,d) = size(X) # n samples in R^d
    # initialize
    θ = randn(m, d)
    θ = θ ./ sqrt.(sum(θ.^2, dims=2)) # random neurons on the sphere
    θ[:,1] = cos.(range(0,2π,length=m))
    θ[:,2] = sin.(range(0,2π,length=m))
    a = zeros(m) # initialization
    b = copy(a)
    c = copy(a)
    gradG = copy(a)
    as = zeros(m,niter)
    loss  = zeros(niter)
    act  =  max.( θ * X', 0.0) # activations
    theta = 1.0
    
    @showprogress 1 "Training neural network..." for iter = 1:niter
        as[:,iter] = a
        outa   =  (1/m) * sum( a .* act , dims=1)[:] # predictions of the network
        loss[iter] = (1/2)*sum( ( outa - Y).^2 )/n + lambda * sum(abs.(a))/m
        b = (1-theta)*a .+ theta*c
        outb   =  (1/m) * sum( b .* act , dims=1)[:] # predictions of the network
        gradR = (outb .- Y)/n  # size n
        gradG = act * gradR
        if p>1
            temp = sfth.( sign.(c).*abs.(c).^(p-1) /(p-1) - (stepsize/theta) * gradG, (stepsize/theta)*lambda)
            c    = (p-1)^(1/(p-1)) * sign.(temp) .* abs.(temp).^(1/(p-1))
        else
            c = sinh.(sfth.( asinh.(c) - (stepsize/theta) * gradG, (stepsize/theta)*lambda))
        end
        a = (1-theta)*a .+ theta*c
        theta = (sqrt(theta^2+4)-theta)*theta/2
    end

    return as,θ, loss, gradG
end

In [ ]:
Random.seed!(1)
X = [-1 1; -1/4 1; 1/4 1; 1 1]
Y = [-1; 1/2; -1/2; 1]
n = 10
sigma = 0.8
X = cat(range(-1,1,length=n),ones(n),dims=2)
Y = abs.(X[:,1]) .- 0.5 .+ sigma*( 2*rand(n) .- 1)
m = 2000
stepsize = 1
niter = 200000
asA, θ, lossA, gradGA = neuralnet_PGM(X, Y, m, stepsize, niter, p=1, lambda=0.02)
asB, θ, lossB, gradGB = neuralnet_PGM(X, Y, m, stepsize, niter, p=1.5, lambda=0.02)
asC, θ, lossC, gradGC = neuralnet_PGM(X, Y, m, stepsize, niter, p=2, lambda=0.02)
asD, θ, lossD = neuralnet_PGM(X, Y, m, stepsize, niter, p=4, lambda=0.02)
asAA, θ, lossAA, gradGAA = neuralnet_APGM(X, Y, m, stepsize, niter, p=1, lambda=0.02)
asBB, θ, lossBB, gradGBB = neuralnet_APGM(X, Y, m, stepsize, niter, p=1.5, lambda=0.02)
asCC, θ, lossCC, gradGCC = neuralnet_APGM(X, Y, m, stepsize, niter, p=2, lambda=0.02);

In [ ]:
opt = lossAA[end];

In [ ]:
figure(figsize=[4,3])
loglog(lossA[1:10000] .- opt,"C0",label=L"p=1") 
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossB[1:10000] .- opt,"C1",label=L"p=1.5")
loglog(lossC[1:10000] .- opt,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-1)*((lossA[1000]-opt)/trgy[1]^(-1)) ,"--C0",label=L"k^{-1}")
loglog(trgx, trgy.^(-1/1.5)*((lossB[1000]-opt)/trgy[1]^(-1/1.5)) ,"--C1",label=L"k^{-1/1.5}")
loglog(trgx, trgy.^(-1/2)*((lossC[1000]-opt)/trgy[1]^(-1/2)) ,"--C3",label=L"k^{-1/2}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("NN_PGM.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[4,3])
loglog(lossAA[1:10000] .- opt,"C0",label=L"p=1") 
trgx = 100*[10; 100; 100; 10]
trgy = copy(trgx)
trgy[3] = trgx[1]
loglog(lossBB[1:10000] .- opt,"C1",label=L"p=1.5")
loglog(lossCC[1:10000] .- opt,"C3",label=L"p=2") 
loglog(trgx, trgy.^(-2)*((lossAA[1000]-opt)/trgy[1]^(-2)) ,"--C0",label=L"k^{-2}")
loglog(trgx, trgy.^(-2/1.5)*((lossBB[1000]-opt)/trgy[1]^(-2/1.5)) ,"--C1",label=L"k^{-2/1.5}")
loglog(trgx, trgy.^(-2/2)*((lossCC[1000]-opt)/trgy[1]^(-2/2)) ,"--C3",label=L"k^{-1}")

legend(loc=3,ncol=2)
xlabel(L"k")
ylabel(L"F(f_k)-\inf\; F")
grid("on")
savefig("NN_APGM.png",bbox_inches="tight",dpi=200)

In [ ]:
X_test = cat(range(-1.2,1.2,length=200),ones(200),dims=2 )

figure(figsize=[4,3])
Y_test = (1/m) * sum( asAA[:,100000] .* max.( θ* X_test', 0.0) , dims=1)[:]
plot(X_test[:,1], Y_test,"k",label="solution")
Y_test = (1/m) * sum( asA[:,200] .* max.( θ* X_test', 0.0) , dims=1)[:]
plot(X_test[:,1], Y_test,"C0",label=L"p=1")
Y_test = (1/m) * sum( asB[:,200] .* max.( θ* X_test', 0.0) , dims=1)[:]
plot(X_test[:,1], Y_test,"C1",label=L"p=1.5")
Y_test = (1/m) * sum( asC[:,200] .* max.( θ* X_test', 0.0) , dims=1)[:]
plot(X_test[:,1], Y_test,"C3",label=L"p=2")
Y_test = (1/m) * sum( asD[:,200] .* max.( θ* X_test', 0.0) , dims=1)[:]
plot(X_test[:,1], Y_test,"C4",label=L"p=4")


plot(X[:,1],Y,"ko",ms=2,label="samples")
legend(ncol=2)
xticks([-1,0,1]);xlabel(L"x")
yticks([-1,0,1]);ylabel(L"y")
axis([-1,1,-1,1])
grid("on")
savefig("NN_PGM_regressor_all.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[14,2.5])
cmap = ColorMap("viridis")
ts = (6).^(0:4)
θ_init =range(0,2π,length=m)
subplot(141)
for t=1:length(ts)
    fill_between(θ_init,asA[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
axis([0,2π,-6,22])
axis("off")
ylabel(L"Density of $f_k$")
title(L"p=1")

subplot(142)
for t=1:length(ts)
    fill_between(θ_init,asB[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
axis([0,2π,-6,22])
title(L"p=1.5")
axis("off")
subplot(143)
for t=1:length(ts)
    fill_between(θ_init,asC[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
axis([0,2π,-6,22])
title(L"p=2")
axis("off")
subplot(144)
for t=1:length(ts)
    fill_between(θ_init,asD[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
axis([0,2π,-6,22])
title(L"p=4")
axis("off")
savefig("NN_PGM_measures_all.png",bbox_inches="tight",dpi=200)

In [ ]:
figure(figsize=[10,3])
cmap = ColorMap("viridis")
ts = (6).^(0:5)
θ_init =range(0,2π,length=m)
subplot(121)
for t=1:length(ts)
    fill_between(θ_init,asA[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
plot(θ_init,500*gradGA,"k",label=L"\bar G'[\mu^*]")
axis([0,2π,-14,38])
axis("off")
legend()
ylabel(L"Density of $f_k$")
title(L"p=1")

subplot(122)
for t=1:length(ts)
    fill_between(θ_init,asC[:,ts[t]],color=cmap(1.3-t/length(ts)),alpha=0.4)
end
plot(θ_init,500*gradGA,"k")
axis([0,2π,-14,38])
title(L"p=2")
axis("off")
savefig("NN_PGM_measures_grad.png",bbox_inches="tight",dpi=200)